In [69]:


# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))



In [70]:
print("HELLO")

HELLO


In [71]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.datasets import mnist
import numpy as np
import os

In [72]:
print("Priyanshi")

Priyanshi


In [73]:
import tensorflow as tf
from tensorflow.keras import layers

def make_discriminator_model():
    model = tf.keras.Sequential()

    # First Conv2D layer
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', 
                            kernel_initializer='he_normal', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Second Conv2D layer
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same',
                            kernel_initializer='he_normal'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Flatten and Dense output
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model


In [74]:
# def make_discriminator_model():
#     model = tf.keras.Sequential()

#     model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[128, 128, 1]))
#     model.add(layers.LeakyReLU())
#     model.add(layers.Dropout(0.3))

#     model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
#     model.add(layers.LeakyReLU())
#     model.add(layers.Dropout(0.3))

#     model.add(layers.Flatten())
#     model.add(layers.Dense(1, activation='sigmoid'))

#     return model

In [75]:
(x_train, _), (_, _) = mnist.load_data()

In [76]:
print(x_train)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [77]:
x_train = x_train.astype('float32') / 255.0
print(x_train.shape)

(60000, 28, 28)


In [78]:
x_train = np.expand_dims(x_train, axis=-1) 
print(x_train.shape)

(60000, 28, 28, 1)


In [79]:
indices = np.random.permutation(len(x_train))  # Generate a shuffled set of indices
x_train = x_train[indices] 

In [80]:
num_samples = x_train.shape[0]
half_size = num_samples // 2

In [81]:
x_train_part1 = x_train[:30000]  # First 30k images
x_train_part2 = x_train[30000:60000]  # Next 30k images

# Create labels
y_train_part1 = np.ones(len(x_train_part1), dtype=int)  # Labels for first part: 1
y_train_part2 = np.zeros(len(x_train_part2), dtype=int)  # Labels for second part: 0

# Combine into a single dataset if needed
x_train_combined = np.concatenate([x_train_part1, x_train_part2], axis=0)
y_train_combined = np.concatenate([y_train_part1, y_train_part2], axis=0)

In [82]:
# x_label_1 = x_train[:half_size]
# x_label_0 = x_train[half_size:]



In [83]:
# y_label_1 = np.ones((half_size, 1))
# y_label_0 = np.zeros((half_size, 1))


In [84]:
# x_combined = np.vstack((x_label_1, x_label_0))
# y_combined = np.vstack((y_label_1, y_label_0))

In [85]:
indices = np.arange(x_train_combined.shape[0])
np.random.shuffle(indices)

x_combined = x_train_combined[indices]
y_combined = y_train_combined[indices]

In [86]:
print(x_combined.shape)

(60000, 28, 28, 1)


In [87]:
discriminator = make_discriminator_model()
discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss=BinaryCrossentropy(), metrics=['accuracy'])

In [88]:
import tensorflow as tf

print("Is GPU available:", tf.config.list_physical_devices('GPU'))


Is GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [89]:
import os
import tensorflow as tf

# Training setup
batch_size = 256
epochs = 100
save_dir = "discriminator_weights"
os.makedirs(save_dir, exist_ok=True)

# Ensure data normalization and correct shapes
# x_combined = x_combined / 255.0  # Normalize to [0, 1]

# Training loop
with tf.device('/GPU:0'):  # Explicitly run on GPU
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        # Train the model for one epoch
        history = discriminator.fit(
            x_combined, y_combined,
            batch_size=batch_size,
            epochs=1,
            verbose=1,  # Displays training progress
            shuffle=True  # Shuffles data at the start of each epoch
        )

        # Display probabilities for the first 5 samples
        probs = discriminator.predict(x_combined[:5], verbose=0).flatten()
        print(f"Probabilities (first 5): {probs}")

        # Save weights periodically (every 10 epochs)
        if (epoch + 1) % 10 == 0:
            weight_path = os.path.join(save_dir, f"epoch_{epoch+1}.weights.h5")
            discriminator.save_weights(weight_path)
            print(f"Saved weights for epoch {epoch+1}: {weight_path}")



Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.5040 - loss: 0.7302
Probabilities (first 5): [0.51928365 0.5532865  0.5190712  0.4951808  0.5254803 ]

Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5066 - loss: 0.7060
Probabilities (first 5): [0.4187516  0.501759   0.48407984 0.46183267 0.44085315]

Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5089 - loss: 0.7013
Probabilities (first 5): [0.5397753  0.5140789  0.48892012 0.49722078 0.5074379 ]

Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5092 - loss: 0.6990
Probabilities (first 5): [0.51086706 0.5808643  0.5115778  0.52560747 0.5364907 ]

Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5158 - loss: 0.6966
Probabilities (first 5): [0.47880378 0.5500528  0.49670526 0.4911652  0.4739305 ]

Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5142 - loss: 0.6953
Probabilities (first 5): [0.52077174 0.56579965 0.53964126 0.

In [90]:
(_, _), (x_test, _) = mnist.load_data()

In [91]:
print(x_test.shape)

(10000, 28, 28)


In [92]:
# sample_index = 200 # Index of the sample you want to predict
# prediction = discriminator.predict(np.expand_dims(x_test[sample_index], axis=0))  # Add batch dimension
# print(prediction)


In [106]:
x_test_normalized = x_test / 255.0

sample_index = 145

# Check the shape of the sample
print("Original shape:", x_test[sample_index].shape)

# Reshape the sample if needed
sample_input = np.expand_dims(x_test_normalized[sample_index], axis=0)  # Add batch dimension
if len(sample_input.shape) == 3:  # Missing channel dimension
    sample_input = np.expand_dims(sample_input, axis=-1)  # Add channel dimension

print("Reshaped input shape:", sample_input.shape)

# Make prediction
prediction = discriminator.predict(sample_input)
print(f"Prediction: {prediction}")


Original shape: (28, 28)
Reshaped input shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Prediction: [[0.5005083]]


In [105]:
x_test_normalized = x_test / 255.0

sample_index = 94

# Check the shape of the sample
print("Original shape:", x_test[sample_index].shape)

# Reshape the sample if needed
sample_input = np.expand_dims(x_test_normalized[sample_index], axis=0)  # Add batch dimension
if len(sample_input.shape) == 3:  # Missing channel dimension
    sample_input = np.expand_dims(sample_input, axis=-1)  # Add channel dimension

print("Reshaped input shape:", sample_input.shape)

# Make prediction
prediction = discriminator.predict(sample_input)
print(f"Prediction: {prediction}")


Original shape: (28, 28)
Reshaped input shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Prediction: [[0.49896276]]


In [108]:
def test_discriminator(discriminator, image_1, image_2):
    """
    Tests the discriminator with two images and returns their predicted probabilities.
    
    Parameters:
    - discriminator: The trained discriminator model.
    - image_1: The first input image (should be a single 28x28x1 numpy array).
    - image_2: The second input image (should be a single 28x28x1 numpy array).
    
    Returns:
    - prob_1: Predicted probability for image_1.
    - prob_2: Predicted probability for image_2.
    """
    # Ensure images are properly reshaped to include batch dimension
    if len(image_1.shape) == 3:  
        image_1 = np.expand_dims(image_1, axis=0) 
    if len(image_2.shape) == 3: 
        image_2 = np.expand_dims(image_2, axis=0)  
    
    # Predict probabilities
    prob_1 = discriminator.predict(image_1, verbose=0)[0][0]
    prob_2 = discriminator.predict(image_2, verbose=0)[0][0]
    
    
    return prob_1, prob_2

# Example: Test with two random images
test_image_1 = x_train_part1[50]  # Label 1
test_image_2 = x_train_part2[25]  # Label 0
prob_1, prob_2 = test_discriminator(discriminator, test_image_1, test_image_2)
print(f"\nProbability for Image 1 (label 1): {prob_1:.4f}")
print(f"Probability for Image 2 (label 0): {prob_2:.4f}")


Probability for Image 1 (label 1): 0.5671
Probability for Image 2 (label 0): 0.5166
